# Classification With TensorFlow

### Introduction

In this notebook, we will be classifying 65 different types of fruits using just over 43,000 images.  The fruits were planted in the shaft of a low speed motor (3 rpm) and a short movie of 20 seconds was recorded. A Logitech C920 camera was used for filming the fruits. This is one of the best webcams available. Behind the fruits we placed a white sheet of paper as background.  This dataset comes from kaggle.com https://www.kaggle.com/moltean/fruits

### Basic Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras as ks
from skimage import io
import os
import csv

C:\Users\etallen127\AppData\Local\Continuum\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Functions

In [6]:
# Importing cv2 to help work with images
import cv2

# Creating a function to format images
def load_images(path):
    img_data = [] # return the image itself
    labels = [] # adds a label based on the directory
    index = [] # adds an index to reference image
    x = -1
    for pic in os.listdir(path):
        pic_path = os.path.join(path,pic)
        labels.append(pic)
        x = x+1
        for img in os.listdir(pic_path):
            img_path = os.path.join(pic_path,img)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (64, 64))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            img_data.append(image)
            index.append(x)
    return np.array(img_data),np.array(index),labels

### Shaping Data

In [7]:
# Setting up fill location for our load_images function
train_path = 'fruits-360_dataset_2018_06_03/fruits-360/Training'
valid_path = 'fruits-360_dataset_2018_06_03/fruits-360/Validation'

In [10]:
# Calling function and creating train and test data
X_train,y_train,label_data = load_images(train_path)
X_test,y_test,label_data_garbage = load_images(valid_path)

In [13]:
# Making Categorical
num_of_classes = 65
Y_train = ks.utils.np_utils.to_categorical(y_train, num_of_classes)
Y_test = ks.utils.np_utils.to_categorical(y_test, num_of_classes)

### Modeling

In [36]:
# Importing Modeling Tools
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization

In [39]:
# Init Model
model = Sequential()

In [40]:
model.add(Conv2D(16, (3, 3), input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3 )))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(Dense(65))

model.add(Activation('softmax'))

In [41]:
# Compiling Model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

In [42]:
# Training Model
model.fit(X_train,Y_train, epochs=5,batch_size = 32,
                    validation_data=(X_test,Y_test))

Train on 32426 samples, validate on 10903 samples
Epoch 1/5
32426/32426 [==============================] - 844s 26ms/step - loss: 0.7395 - acc: 0.7950 - val_loss: 0.7178 - val_acc: 0.8120
Epoch 2/5
32426/32426 [==============================] - 827s 25ms/step - loss: 0.1679 - acc: 0.9447 - val_loss: 0.2699 - val_acc: 0.9142
Epoch 3/5
32426/32426 [==============================] - 822s 25ms/step - loss: 0.1138 - acc: 0.9628 - val_loss: 0.1713 - val_acc: 0.9483
Epoch 4/5
32426/32426 [==============================] - 825s 25ms/step - loss: 0.0925 - acc: 0.9692 - val_loss: 0.1523 - val_acc: 0.9582
Epoch 5/5
32426/32426 [==============================] - 828s 26ms/step - loss: 0.0782 - acc: 0.9742 - val_loss: 0.2062 - val_acc: 0.9435


In [43]:
model.evaluate(X_test, Y_test)

10903/10903 [==============================] - 65s 6ms/step


[0.2061912194303365, 0.9435017884985784]

# Conclusion